In [1]:
!cat .env

STAGE=CCR_PRODUCTION


In [2]:
Company.objects.all()

<QuerySet [<Company: 3af64f25-59e8-446b-bb1e-963549090b0d: CCR - Rio Sp>]>

In [3]:
import json
import os
from openpyxl import load_workbook
from datetime import timedelta
import random
from copy import deepcopy
from django.contrib.gis.geos import LineString, Point
from django.contrib.admin.utils import flatten
import math
import datetime
import locale
from tqdm.notebook import tqdm
from PIL import Image
from io import BytesIO
from django.contrib.gis.geos import Point, LineString as DjangoLineString
import pytz

In [4]:
from simple_history.utils import bulk_create_with_history
from helpers.histories import bulk_update_with_history
# from openpyxl_image_loader import SheetImageLoader

In [5]:
from helpers.route_maker import dic_to_ordered_list, unequal_point_pairs

In [6]:
def km_to_coordinates(road, km):

    road_marks = dic_to_ordered_list(road.marks)
    selected_pair = None

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] > km and pair[1]["km"] < km:
            selected_pair = pair
            break
        elif pair[1]["km"] > km and pair[0]["km"] < km:
            selected_pair = pair
            break
        elif pair[0]["km"] == km:
            return Point(pair[0]["point"]["coordinates"]), road
        elif pair[1]["km"] == km:
            return Point(pair[1]["point"]["coordinates"]), road

    # Calculate total length
    start_km = min([selected_pair[0]["km"], selected_pair[1]["km"]])
    end_km = max([selected_pair[0]["km"], selected_pair[1]["km"]])
    segment_mark_length = end_km - start_km
    distance_from_min = km - start_km
    # Check if km is decreasing
    invert_km = selected_pair[1]["km"] <= selected_pair[0]["km"]

    # Cut segment
    start_key = min((selected_pair[0]["index"], selected_pair[1]["index"]))
    end_key = max((selected_pair[0]["index"], selected_pair[1]["index"]))

    points = []
    for key in range(start_key, end_key + 1):
        points.append(
            Point(road.path[key][0], road.path[key][1], road.path[key][2])
        )

    segment = LineString(points, srid=4326)
    # Apply km inversion correction
    if invert_km:
        segment_distance = (
            1 - (distance_from_min / segment_mark_length)
        ) * segment.length
    else:
        segment_distance = (
            distance_from_min / segment_mark_length
        ) * segment.length
    # Find point
    point = segment.interpolate(segment_distance)

    return Point(point.x, point.y), road

In [7]:
def check_valid_road(road, km):

    road_marks = dic_to_ordered_list(road.marks)

    for pair in unequal_point_pairs(road_marks):
        if pair[0]["km"] >= km and pair[1]["km"] <= km:
            return True
        elif pair[1]["km"] >= km and pair[0]["km"] <= km:
            return True

    return False

In [8]:
def get_road_coordinates(road_name, km, direction, company):
    road_set = Road.objects.filter(
        name__contains=road_name, direction=int(direction), company=company
    )
#     print(road_set)

    # If roads in specified direction are not found, search road only
    # by name and order then by direction
    road_set_generic = Road.objects.filter(
        name__contains=road_name, company=company
    ).order_by("direction")

    # Check if KM range in road_set
#     print("Searching KM on direction...")
    valid = False
    for road in road_set:
        if check_valid_road(road, km):
#             print("Found KM on Road {}".format(road))
            valid = True
            break

    if not valid:
#         print("Searching KM without direction...")
        for road in road_set_generic:
            if check_valid_road(road, km):
#                 print("Found KM on Road {}".format(road))
                valid = True
                break

    if not valid:
        return Point(0, 0), None

    try:
        return km_to_coordinates(road, km)
    except Exception:
        return Point(0, 0), None

In [9]:
def get_connected_reference(company, resource, key, value):
    value=value.strip()
    select_options = company.custom_options[resource]['fields'][key]['selectOptions']['options']
    return next(a['value'] for a in select_options if a['name'].lower() == value.lower())

In [10]:
def get_value(field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['selectOptions']['options'] for item in values if item['displayName']==field])
#     print(item_translation)
    if field == 'Nome da placa':
        value=value.upper()
    final_translation = {item['name']: item['value'] for item in item_translation}
#     print(final_translation)
    return final_translation[value]

In [11]:
def get_sign(field, sign_field, occ, value):
    values = occ.form_fields['fields']
    item_translation = flatten([item['innerFields'] for item in values if item['displayName']==field])
#     print(item_translation)
    mid_translation = flatten([item['selectOptions']['options'] for item in item_translation if item['displayName'] == sign_field])
#     print(mid_translation)
    final_translation = {item['name']: item['value'] for item in mid_translation}
#     print(final_translation)
    return final_translation[value]

In [12]:
!cat .env

STAGE=CCR_PRODUCTION


In [32]:
filename='TERR_BR116 RJ (02-08-23)'

In [33]:
wb = load_workbook(filename + '.xlsx')
sheetname = wb.sheetnames[0]
# sheetname
ws = wb[sheetname]

In [34]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
#     if obj['Rodovia'] is not None:           
#         obj['img'] = [a for a in ws._images if a.anchor._from.row == index]
#         for i, img in enumerate(obj['img']):
#             if i == 0:
#                 obj['img'][i].name = obj['Descrição Foto Contenção 1_1']
#                 obj['img'][i].tipo = obj['Tipo Foto Contenção 1_1']
#             elif i == 1:
#                 obj['img'][i].name = obj['Descrição Foto Contenção 1_2']
#                 obj['img'][i].tipo = obj['Tipo Foto Contenção 1_2']
#             elif i == 2:
#                 obj['img'][i].name = obj['Descrição Foto Contenção 2_1']
#                 obj['img'][i].tipo = obj['Tipo Foto Contenção 2_1']
#             elif i == 3:
#                 obj['img'][i].name = obj['Descrição Foto Contenção 2_2']
#                 obj['img'][i].tipo = obj['Tipo Foto Contenção 2_2']
#             elif i == 4:
#                 obj['img'][i].name = obj['Descrição Foto Contenção 3_1']
#                 obj['img'][i].tipo = obj['Tipo Foto Contenção 3_1']
#             elif i == 5:
#                 obj['img'][i].name = obj['Descrição Foto Contenção 3_2']
#                 obj['img'][i].tipo = obj['Tipo Foto Contenção 3_2']
#             elif i == 6:
#                 obj['img'][i].name = obj['Descrição Foto Contenção 4_1']
#                 obj['img'][i].tipo = obj['Tipo Foto Contenção 4_1']
#             elif i == 7:
#                 obj['img'][i].name = obj['Descrição Foto Contenção 4_2']
#                 obj['img'][i].tipo = obj['Tipo Foto Contenção 4_2']

        
    values.append(obj)

In [35]:
for a in values:
    a['km'] = a['km'].replace('+','.')
    a['km final'] = a['km final'].replace('+','.')
    for k,b in a.items():
        if b == '-' or b == ' -' or b == ' - ':
            a[k] = ''
        if b == 'TELA' or b == 'TELA ' or b == ' TELA ' or b == ' TELA':
            a[k] = 'Tela'
        if b == 'OUTRO' or b == 'outro':
            a[k] = 'Outro'
        if b == ' TELA GRAMPEADA' and k == 'Tipo de Estrutura_3':
            a[k] = 'Tela Grampeada'
        if b == 'Muro De Concreto Por Gravidade':
            a[k] = 'Muro de Concreto por Gravidade'
        if b == 'CORTINA ATIRANTADA' or b == ' CORTINA ATIRANTADA' or b == ' CORTINA ATIRANTADA ':
            a[k] = 'Cortina Atirantada'
        if b == 'TERRA ARMADA':
            a[k] = 'Terra Armada'
        if b == 'CONCRETO PROJETADO SIMPLES':
            a[k] = 'Concreto Projetado Simples'
        if b == 'CRIB-WALLS ' or b == ' CRIB-WALLS':
            a[k] = 'Crib-Walls'
        if k == 'Ancoragens da Contenção_1' and (b == 'Tirantes' or b == 'Tirantes ' or b == ' Tirantes '):
            a[k] = 'Tirante'
        if b == 'ARBÓREA / Arbustiva':
            a[k] = 'Arbórea'

In [36]:
values[138]

{'km': '232.800',
 'km final': '233.000',
 'Status': 'Identificado',
 'Equipe/Empreiteira': 'Civil: SGTEC',
 'Encontrado em': datetime.datetime(2022, 10, 20, 0, 0),
 'Sentido': 'NORTE',
 'Classe': 'Terraplenos e Estrutura de Contenção',
 'Faixa': 'Não se aplica',
 'Rodovia': 'BR-116 RJ',
 'Latitude \n(Grau decimal - 00,00000)': 619293,
 'Longitude  \n(Grau decimal - 00,00000)': 7492795,
 'X UTM': -43.83884,
 'Y UTM': -22.66766,
 'Zona UTM': None,
 'Identificação': 'TP 116 RJ 232+800 N 1',
 'UF': 'RJ',
 'Extensão Terrapleno (m)': '200',
 'Altura Terrapleno (m)': '80',
 'Inclinação Terrapleno (°)': '50',
 'Distância Acostamento Terrapleno (m)': '0',
 'Tipo Terrapleno': 'ATERRO',
 'Tipo de Relevo': 'MONTANHOSO',
 'Vegetação': 'ARBÓREA',
 'Densidade da Vegetação': 'MÉDIA',
 'Drenagem Superficial': 'INEXISTENTE',
 'Drenagem Subterrânea': 'INEXISTENTE',
 'Tipo de Drenagem Subterrânea': '',
 'Passivo Ambiental': 'SIM',
 'Descrição do Passivo Ambiental': 'BR116-RJ-PA-021',
 'Observações': '',


In [19]:
company = Company.objects.get(name='CCR - Rio Sp')

occurrence_type=OccurrenceType.objects.get(uuid='62b15386-f114-44ff-8879-a561fbe1c460')
user=User.objects.get(username='rlcs')
utc = pytz.timezone('America/Sao_Paulo')
company,occurrence_type

(<Company: 3af64f25-59e8-446b-bb1e-963549090b0d: CCR - Rio Sp>,
 <OccurrenceType: Terraplenos e Estrutura de Contenção - ['CCR - Rio Sp']>)

In [17]:
for a in tqdm(values):
    a['contencao'] = list(filter(None, [{'structure_kind': get_sign('Dados da Estrutura de Contenção','Tipo de Estrutura',occurrence_type,a['Tipo de Estrutura_1'].strip()) if 'Tipo de Estrutura_1' in a and a['Tipo de Estrutura_1'] is not None and a['Tipo de Estrutura_1']!='' else None, 'other': a['Outro tipo de estrutura_1'] if a['Tipo de Estrutura_1'] == 'Outro' else None, 'extenstion': a['Extensão Contenção (m)_1'], 'inner_height': a['Altura Contenção (m)_1'],'anchor':get_sign('Dados da Estrutura de Contenção','Ancoragens da Contenção',occurrence_type,a['Ancoragens da Contenção_1']) if 'Ancoragens da Contenção_1' in a and a['Ancoragens da Contenção_1'] is not None and a['Ancoragens da Contenção_1'] != '' else None, 'inner_quantity': a['Quantidade (unidade)_1'] if 'Quantidade (unidade)_1' in a else None} if a['Tipo de Estrutura_1'] is not None and a['Tipo de Estrutura_1'] != '' else None,
                                    {'structure_kind': get_sign('Dados da Estrutura de Contenção','Tipo de Estrutura',occurrence_type,a['Tipo de Estrutura_2'].strip()) if 'Tipo de Estrutura_2' in a and a['Tipo de Estrutura_2'] is not None and a['Tipo de Estrutura_2']!='' else None, 'other': a['Outro tipo de estrutura_2'] if a['Tipo de Estrutura_2'] == 'Outro' else None, 'extenstion': a['Extensão Contenção (m)_2'], 'inner_height': a['Altura Contenção (m)_2'],'anchor':get_sign('Dados da Estrutura de Contenção','Ancoragens da Contenção',occurrence_type,a['Ancoragens da Contenção_2']) if 'Ancoragens da Contenção_2' in a and a['Ancoragens da Contenção_2'] is not None and a['Ancoragens da Contenção_2'] != '' else None, 'inner_quantity': a['Quantidade (unidade)_2'] if 'Quantidade (unidade)_2' in a else None} if 'Tipo de Estrutura_2' in a and a['Tipo de Estrutura_2'] is not None and a['Tipo de Estrutura_2'] != '' else None,
                                    {'structure_kind': get_sign('Dados da Estrutura de Contenção','Tipo de Estrutura',occurrence_type,a['Tipo de Estrutura_3'].strip()) if 'Tipo de Estrutura_3' in a and a['Tipo de Estrutura_3'] is not None and a['Tipo de Estrutura_3']!='' else None, 'other': a['Outro tipo de estrutura_3'] if a['Tipo de Estrutura_3'] == 'Outro' else None, 'extenstion': a['Extensão Contenção (m)_3'], 'inner_height': a['Altura Contenção (m)_3'],'anchor':get_sign('Dados da Estrutura de Contenção','Ancoragens da Contenção',occurrence_type,a['Ancoragens da Contenção_3']) if 'Ancoragens da Contenção_3' in a and a['Ancoragens da Contenção_3'] is not None and a['Ancoragens da Contenção_3'] != '' else None, 'inner_quantity': a['Quantidade (unidade)_3'] if 'Quantidade (unidade)_3' in a else None} if 'Tipo de Estrutura_3' in a and a['Tipo de Estrutura_3'] is not None and a['Tipo de Estrutura_3'] != '' else None,
                                    {'structure_kind': get_sign('Dados da Estrutura de Contenção','Tipo de Estrutura',occurrence_type,a['Tipo de Estrutura_4'].strip()) if 'Tipo de Estrutura_4' in a and a['Tipo de Estrutura_4'] is not None and a['Tipo de Estrutura_4']!='' else None, 'other': a['Outro tipo de estrutura_4'] if a['Tipo de Estrutura_4'] == 'Outro' else None, 'extenstion': a['Extensão Contenção (m)_4'], 'inner_height': a['Altura Contenção (m)_4'],'anchor':get_sign('Dados da Estrutura de Contenção','Ancoragens da Contenção',occurrence_type,a['Ancoragens da Contenção_4']) if 'Ancoragens da Contenção_4' in a and a['Ancoragens da Contenção_4'] is not None and a['Ancoragens da Contenção_4'] != '' else None, 'inner_quantity': a['Quantidade (unidade)_4'] if 'Quantidade (unidade)_4' in a else None} if 'Tipo de Estrutura_4' in a and a['Tipo de Estrutura_4'] is not None and a['Tipo de Estrutura_4'] != '' else None,
                                    {'structure_kind': get_sign('Dados da Estrutura de Contenção','Tipo de Estrutura',occurrence_type,a['Tipo de Estrutura_5'].strip()) if 'Tipo de Estrutura_5' in a and a['Tipo de Estrutura_5'] is not None and a['Tipo de Estrutura_5']!='' else None, 'other': a['Outro tipo de estrutura_5'] if a['Tipo de Estrutura_5'] == 'Outro' else None, 'extenstion': a['Extensão Contenção (m)_5'], 'inner_height': a['Altura Contenção (m)_5'],'anchor':get_sign('Dados da Estrutura de Contenção','Ancoragens da Contenção',occurrence_type,a['Ancoragens da Contenção_5']) if 'Ancoragens da Contenção_5' in a and a['Ancoragens da Contenção_5'] is not None and a['Ancoragens da Contenção_5'] != '' else None, 'inner_quantity': a['Quantidade (unidade)_5'] if 'Quantidade (unidade)_5' in a else None} if 'Tipo de Estrutura_5' in a and a['Tipo de Estrutura_5'] is not None and a['Tipo de Estrutura_5'] != '' else None]))

  0%|          | 0/520 [00:00<?, ?it/s]

In [18]:
values[11]

{'km': '216.800',
 'km final': '216.900',
 'Status': 'Identificado',
 'Equipe/Empreiteira': 'Civil: SGTEC',
 'Encontrado em': datetime.datetime(2022, 10, 21, 0, 0),
 'Sentido': 'NORTE',
 'Classe': 'Terraplenos e Estrutura de Contenção',
 'Faixa': 'Não se aplica',
 'Rodovia': 'BR-116 RJ',
 'Latitude \n(Grau decimal - 00,00000)': 628552,
 'Longitude  \n(Grau decimal - 00,00000)': 7488887,
 'X UTM': -43.748411,
 'Y UTM': -22.702277,
 'Zona UTM': None,
 'Identificação': 'TP 116 RJ 216+800 N 1',
 'UF': 'RJ',
 'Extensão Terrapleno (m)': '100',
 'Altura Terrapleno (m)': '90 /60',
 'Inclinação Terrapleno (°)': '10',
 'Distância Acostamento Terrapleno (m)': '0',
 'Tipo Terrapleno': 'CORTE',
 'Tipo de Relevo': 'MONTANHOSO',
 'Vegetação': 'ARBUSTIVA  ',
 'Densidade da Vegetação': 'ALTA',
 'Drenagem Superficial': 'INEXISTENTE',
 'Drenagem Subterrânea': 'INEXISTENTE',
 'Tipo de Drenagem Subterrânea': '',
 'Passivo Ambiental': 'NÃO',
 'Descrição do Passivo Ambiental': 'NÃO',
 'Observações': '',
 'Ex

In [19]:
objects=[]

for index, a in enumerate(tqdm(values)):

    point, road = get_road_coordinates(a['Rodovia'], float(a['km'].replace('+','.')), str(get_connected_reference(company, 'reporting', 'direction', a['Sentido'].capitalize())) , company)
    if road.name != a['Rodovia']:
        print(road.name,a['Rodovia'])
        break
    try:
        objects.append(Reporting(
                    company=company,
                    occurrence_type=occurrence_type,
                    lane=get_connected_reference(company, 'reporting', 'lane', a['Faixa']),
                    road=road,
                    road_name=road.name,
                    direction=get_connected_reference(company, 'reporting', 'direction', a['Sentido'].capitalize()),
                    created_by=user,
                    found_at=a['Encontrado em'].replace(tzinfo=utc),
                    km=float(a['km']),
                    end_km_manually_specified= True,
                    end_km=float(a['km final']),
                    point=point,
                    form_data={
                        'latitude': a['Latitude \n(Grau decimal - 00,00000)'],
                        'longitude': a['Longitude  \n(Grau decimal - 00,00000)'],
                        'x_utm': a['X UTM'],
                        'y_utm': a['Y UTM'],
                        'zona_utm': a['Zona UTM'],
                        'id_ccr_antt': a['Identificação'],
                        'uf': a['UF'],
                        'length': a['Extensão Terrapleno (m)'],
                        'height_terrapleno': a['Altura Terrapleno (m)'],
                        'inclination': a['Inclinação Terrapleno (°)'],
                        'distance': a['Distância Acostamento Terrapleno (m)'],
                        'tipo_terrapleno': get_value('Tipo Terrapleno',occurrence_type,a['Tipo Terrapleno'].title()) if a['Tipo Terrapleno'] is not None else None,
                        'terrain_embossing': get_value('Tipo de Relevo',occurrence_type,a['Tipo de Relevo'].title().strip()) if a['Tipo de Relevo'] is not None else None,
                        'vegetation': get_value('Vegetação',occurrence_type,a['Vegetação'].title().strip()) if a['Vegetação'] is not None else None,
                        'vegetation_density': get_value('Densidade da Vegetação',occurrence_type,a['Densidade da Vegetação'].title().strip()) if a['Densidade da Vegetação'] is not None else None,
                        'is_contention': True if a['Existe contenção? '].lower() == 'sim' else False,
                        'contention': a['contencao'],
                        'drenagem_superficial': get_value('Drenagem Superficial',occurrence_type,a['Drenagem Superficial'].title()) if a['Drenagem Superficial'] is not None else None,
                        'dre_kind_sup': get_value('Tipo de Drenagem Superficial',occurrence_type,a['Tipo de Drenagem Superficial']) if 'Tipo de Drenagem Superficial' in a and a['Tipo de Drenagem Superficial'] is not None else None,
                        'drenagem_subterranea': get_value('Drenagem Subterrânea',occurrence_type,a['Drenagem Subterrânea'].title()) if 'Drenagem Subterrânea' in a and a['Drenagem Subterrânea'] is not None else None,
                        'kind_drenagem_subterranea': get_value('Tipo de Drenagem Subterrânea',occurrence_type,a['Tipo de Drenagem Subterrânea']) if a['Tipo de Drenagem Subterrânea'] is not None and a['Tipo de Drenagem Subterrânea'] != '' else None,
                        'passivo_ambiental': True if a['Passivo Ambiental'].lower() == 'sim' else False,
                        'environment_description': a['Descrição do Passivo Ambiental'],
                        'notes': a['Observações'],
                        'index': a['Índice (Serial ANTT)'] if 'Índice (Serial ANTT)' in a else ''
                        },
                    ))
    except Exception as e:
        print(f'AQUI: km: {a["km"]} | Rodovia: {a["Rodovia"]} | Linha da Planilha: {index+2} {objects.form_data}| {e}')

  0%|          | 0/520 [00:00<?, ?it/s]

In [22]:
objects[4].__dict__

{'_state': <django.db.models.base.ModelState at 0x7f19937b6bd0>,
 'uuid': UUID('e696f7a8-1a3e-4c72-855f-c2fc7de6bb0f'),
 'number': '',
 'company_id': UUID('3af64f25-59e8-446b-bb1e-963549090b0d'),
 'road_name': 'BR-116 RJ',
 'road_id': 2037,
 'km': 214.78,
 'end_km': 215.78,
 'km_reference': None,
 'project_km': 0,
 'project_end_km': None,
 'point': <Point object at 0x7f19937592e0>,
 'direction': '0',
 'lane': '99',
 'track': None,
 'branch': None,
 'address': {},
 'created_by_id': UUID('508a66f3-3f58-4c4e-bc9a-fbba6078c02d'),
 'firm_id': None,
 'occurrence_type_id': UUID('62b15386-f114-44ff-8879-a561fbe1c460'),
 'form_data': {'latitude': 629945,
  'longitude': 7487705,
  'x_utm': -43.734753,
  'y_utm': -22.712846,
  'zona_utm': None,
  'id_ccr_antt': 'TP 116 RJ 214+780 N 1',
  'uf': 'RJ',
  'length': '1000',
  'height_terrapleno': '80',
  'inclination': '35',
  'distance': '0',
  'tipo_terrapleno': '2',
  'terrain_embossing': '1',
  'vegetation': '1',
  'vegetation_density': '3',
  'is

In [23]:
for reporting in tqdm(objects):
    reporting.delete()
    print(reporting.number)

  0%|          | 0/520 [00:00<?, ?it/s]

'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20050
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20051
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20052
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20053
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20054
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20055
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20

CCR-Inv-2023.20107
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20108
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20109
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20110
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20111
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20112
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20113
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20114
'NoneType' 

CCR-Inv-2023.20165
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20166
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20167
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20168
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20169
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20170
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20171
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without directio

Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20223
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20224
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20225
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20226
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20227
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20228
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20229
'NoneType' object has no attribute 'order'
Searching KM on direction...
S

CCR-Inv-2023.20281
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20282
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20283
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20284
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20285
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20286
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20287
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2037: BR-116 RJ
CCR-Inv-2023.20288
'NoneType' 

CCR-Inv-2023.20340
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.20341
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.20342
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.20343
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.20344
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.20345
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.20346
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on R

Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.20400
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.20401
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.20402
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.20403
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.20404
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.20405
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.20406
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.20407
'N

Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.20461
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.20462
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.20463
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.20464
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.20465
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.20466
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.20467
'NoneType' object has no attribute 'order'
Searching KM

Searching KM on direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.20518
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.20519
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.20520
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.20521
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.20522
'NoneType' object has no attribute 'order'
Searching KM on direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.20523
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without direction...
Found KM on Road 2: BR-116 RJ
CCR-Inv-2023.20524
'NoneType' object has no attribute 'order'
Searching KM on direction...
Searching KM without d

In [43]:
# for reporting, images in tqdm(objects):
#     for image in reversed(images):
#         reporting_file = ReportingFile(
#             created_by=user,
#             reporting=reporting,
#             description=image.name,
#             km=reporting.km,
#             point=reporting.point
#         )
#         reporting_file.save()
#         image_io = BytesIO(image._data())
#         im = Image.open(image_io)
#         if im.mode in ("RGBA", "P"):
#             im = im.convert("RGB")
#         thumb_io = BytesIO()
#         im.save(thumb_io, format='jpeg', quality=90)
#         reporting_file.upload.save(image.name + '.jpeg', thumb_io)

  0%|          | 0/585 [00:00<?, ?it/s]

In [28]:
direc=company.custom_options['reporting']['fields']['direction']['selectOptions']['options']
direc

[{'name': 'Norte', 'short': 'N', 'value': '0'},
 {'name': 'Sul', 'short': 'S', 'value': '1'},
 {'name': 'Leste', 'short': 'L', 'value': '2'},
 {'name': 'Oeste', 'short': 'S', 'value': '3'},
 {'name': 'Norte/Sul', 'short': 'NS', 'value': '4'},
 {'name': 'Leste/Oeste', 'short': 'LO', 'value': '5'},
 {'name': 'Crescente', 'short': 'C', 'value': '6'},
 {'name': 'Decrescente', 'short': 'D', 'value': '7'},
 {'name': 'Canteiro Central', 'short': 'CC', 'value': '8'},
 {'name': 'Ambos', 'short': 'A', 'value': '9'},
 {'name': 'Transversal', 'short': 'TR', 'value': '10'},
 {'name': 'Não se Aplica', 'short': 'X', 'value': '11'}]

In [27]:
values[0]

{'km': '214.030',
 'km final': '214.610',
 'Status': 'Identificado',
 'Equipe/Empreiteira': 'Civil: SGTEC',
 'Encontrado em': datetime.datetime(2022, 10, 26, 0, 0),
 'Sentido': 'SUL',
 'Classe': 'Terraplenos e Estrutura de Contenção',
 'Faixa': 'Não se aplica',
 'Rodovia': 'BR-116 RJ',
 'Latitude \n(Grau decimal - 00,00000)': 630438.35831,
 'Longitude  \n(Grau decimal - 00,00000)': 7487538.82914199,
 'X UTM': -43.729824,
 'Y UTM': -22.714197,
 'Zona UTM': None,
 'Identificação': 'TP 116 RJ 214+030 S 1',
 'UF': 'RJ',
 'Extensão Terrapleno (m)': '580',
 'Altura Terrapleno (m)': '45',
 'Inclinação Terrapleno (°)': '8',
 'Distância Acostamento Terrapleno (m)': '0',
 'Tipo Terrapleno': 'ATERRO',
 'Tipo de Relevo': 'SUAVE',
 'Vegetação': 'RASTEIRA',
 'Densidade da Vegetação': 'ALTA',
 'Drenagem Superficial': 'CONSTRUÍDA',
 'Drenagem Subterrânea': 'INEXISTENTE',
 'Tipo de Drenagem Subterrânea': '',
 'Passivo Ambiental': 'NÃO',
 'Descrição do Passivo Ambiental': 'NÃO',
 'Observações': '',
 'Ex

In [20]:
user=User.objects.get(username="rlcs")

In [37]:
rep_edit=[]
for a in tqdm(values):
    try:
        rep = Reporting.objects.get(company=company,occurrence_type=occurrence_type,form_data__id_ccr_antt=a['Identificação'],direction=next(x['value'] for x in direc if x['name'] == a['Sentido'].capitalize()))
        rep.found_at = a['Encontrado em']
        rep_edit.append(rep)
    except Exception as e:
        print(a['Identificação'],e)

  0%|          | 0/520 [00:00<?, ?it/s]

In [38]:
rep_edit[0].__dict__

{'_state': <django.db.models.base.ModelState at 0x7fc83d4f83d0>,
 'uuid': UUID('f5addf60-c8c5-4922-9ff3-56a55c13e485'),
 'number': 'CCR-Inv-2023.31666',
 'company_id': UUID('3af64f25-59e8-446b-bb1e-963549090b0d'),
 'road_name': 'BR-116 RJ',
 'road_id': 2037,
 'km': 214.03,
 'end_km': 214.61,
 'km_reference': None,
 'project_km': 0.0,
 'project_end_km': 0.58,
 'point': <Point object at 0x7fc82f46e400>,
 'direction': '1',
 'lane': '99',
 'track': None,
 'branch': None,
 'address': {},
 'created_by_id': UUID('508a66f3-3f58-4c4e-bc9a-fbba6078c02d'),
 'firm_id': None,
 'occurrence_type_id': UUID('62b15386-f114-44ff-8879-a561fbe1c460'),
 'form_data': {'uf': 'RJ',
  'index': '',
  'notes': '',
  'x_utm': -43.729824,
  'y_utm': -22.714197,
  'length': '580',
  'distance': '0',
  'latitude': 630438.35831,
  'zona_utm': None,
  'longitude': 7487538.82914199,
  'contention': [],
  'vegetation': '3',
  'id_ccr_antt': 'TP 116 RJ 214+030 S 1',
  'inclination': '8',
  'dre_kind_sup': None,
  'is_cont

In [39]:
apontamentos = bulk_update_with_history(rep_edit,Reporting, use_django_bulk=True,batch_size=150,user=user)

/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.found_at received a naive datetime (2022-10-26 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.found_at received a naive datetime (2022-10-21 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Reporting.found_at received a naive datetime (2022-10-22 00:00:00) while time zone support is active.
  RuntimeWarning)
/home/luan/.cache/pypoetry/virtualenvs/apps-ZFymp6kh-py3.7/lib/python3.7/site-packages/django/db/models/fields/__init__.py:1427: RuntimeWarning: DateTimeField Repo

In [44]:
Reporting.objects.filter(company=company,occurrence_type=occurrence_type,found_at__gte='2023-8-03').delete()

(850, {'reportings.ReportingFile': 2, 'reportings.Reporting': 848})